<a href="https://colab.research.google.com/github/Pankaj-2003/LLMS-GPT/blob/main/makemore_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [2]:
# read in all the words
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [3]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [4]:
# build the dataset

block_size = 3 # context length: how many characters do we take to predict the next one?
X, Y = [], []
for w in words[:5]:
  print(w)
  context = [0] * block_size
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    # print(X)
    Y.append(ix)
    # print(Y)
    print(''.join(itos[i] for i in context), '--->', itos[ix])
    context = context[1:] + [ix] # crop and append

X = torch.tensor(X)
Y = torch.tensor(Y)

emma
... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
olivia
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
ava
... ---> a
..a ---> v
.av ---> a
ava ---> .
isabella
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .
sophia
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .


In [5]:
X.shape , Y.shape

(torch.Size([32, 3]), torch.Size([32]))

#### Making embeddings for our 27 chars (27,2) , the paper had 27k words which they embedded in a space of 30D , but we will do 2D.
#### And our input to neural net will be indexes.
#### The weight matrix can be C , and input will be index.

In [6]:
# embeddings for each alphabet
C = torch.randn((27,2))

In [7]:
print(C)

tensor([[-1.4439,  1.1567],
        [ 1.1159, -0.6462],
        [ 0.6418,  1.9012],
        [-0.8523,  0.0875],
        [ 0.2125,  0.7854],
        [-1.5265, -0.1111],
        [ 0.9947, -0.9230],
        [-1.4611,  0.0171],
        [-0.6100, -0.9765],
        [ 0.4169,  0.5274],
        [-1.1939, -1.0864],
        [ 1.1611,  1.0109],
        [-1.1356, -0.8078],
        [-0.4741, -0.3695],
        [-0.7445, -0.5061],
        [ 0.1843, -0.3535],
        [ 0.4476,  0.7936],
        [-1.4492, -1.0846],
        [-0.2803,  0.9427],
        [ 0.7990,  0.3775],
        [ 0.1702,  0.8657],
        [ 2.0766,  0.9439],
        [ 0.4455,  1.6540],
        [-0.7044, -0.4678],
        [-0.9202, -1.3079],
        [ 0.6717,  0.9296],
        [-0.2658, -0.9255]])


In [8]:
# indexing
# C[torch.tensor([5,6,7])]
emb_x = C[X]
emb_x.shape

torch.Size([32, 3, 2])

In [9]:
# hidden layer
w1 = torch.rand((6,100))
b1 = torch.rand(100)

In [10]:
h = torch.tanh(emb_x.view(-1,6) @ w1 + b1)

In [11]:
h.shape

torch.Size([32, 100])

In [12]:
w2 = torch.rand((100,27))
b2 = torch.rand(27)

In [13]:
logits = h @ w2 + b2

In [14]:
counts = logits.exp()

In [15]:
prob = counts / counts.sum(1, keepdim=True)

In [17]:
prob.shape

torch.Size([32, 27])

In [20]:
loss = -prob[torch.arange(32) , Y].log().mean()

In [21]:
loss

tensor(4.8841)